In [2]:
pip install protobuf

Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf

file_location = "/home/surchand/projects/TranslateManipuri/processing/test.txt"
out_file = "/home/surchand/projects/TranslateManipuri/processing/test_out.txt"

ImportError: cannot import name 'builder' from 'google.protobuf.internal' (/media2/translateManipuriInstallation/miniconda3/envs/homl3/lib/python3.10/site-packages/google/protobuf/internal/__init__.py)

In [2]:
def inference(file_location, translator_model_name):
    reloaded = tf.saved_model.load(translator_model_name)
    readb = open(file_location, "r")
    writeb = open(out_file, "w+")
    # skip the first rows as it is header
    for i in readb:
        data = i.replace("'", "''")
        print("i = {}".format(i))
        print("data = {}".format(data))
        #predicted = reloaded(data).numpy().decode('ISO-8859-1')
        #print("English{}=>Manipuri{}".format(data, predicted))
        #writeb.write(predicted)
        #bengali_text = translate_bengali_scripts(predicted)
        #writesheet.write(i, 2, bengali_text)
    #writeb.save(out_file)

In [13]:
translator_model_name = "translate_manipuri_model/2"

In [14]:
reloaded = tf.saved_model.load(translator_model_name)

In [15]:
#reloaded.signatures['serving_default'].structured_outputs

{'prediction': TensorSpec(shape=(), dtype=tf.string, name='prediction')}

In [12]:
d = reloaded("He is my son")


ValueError: Python inputs incompatible with input_signature:
  inputs: (
    He is my son)
  input_signature: (
    TensorSpec(shape=(None,), dtype=tf.string, name=None)).

In [16]:
#converter = tf.lite.TFLiteConverter.from_keras_model(translator_model_name)
converter = tf.lite.TFLiteConverter.from_saved_model(translator_model_name)

In [17]:
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter.optimizations = [tf.lite.Optimize.DEFAULT] # optional
converter.target_spec.supported_types = [tf.float16] # optional
converter._experimental_lower_tensor_list_ops = False

In [18]:
tflite_model = converter.convert()
# Save the model.
model_name = "tm_new1.tflite"
with open(model_name, 'wb') as f:
  f.write(tflite_model)

2023-12-13 19:16:42.437026: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2023-12-13 19:16:42.437046: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2023-12-13 19:16:42.437591: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: translate_manipuri_model/4
2023-12-13 19:16:42.500575: I tensorflow/cc/saved_model/reader.cc:89] Reading meta graph with tags { serve }
2023-12-13 19:16:42.500601: I tensorflow/cc/saved_model/reader.cc:130] Reading SavedModel debug info (if present) from: translate_manipuri_model/4
2023-12-13 19:16:42.681628: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:357] MLIR V1 optimization pass is not enabled
2023-12-13 19:16:42.731880: I tensorflow/cc/saved_model/loader.cc:229] Restoring SavedModel bundle.
2023-12-13 19:16:43.644843: I tensorflow/cc/saved_model/loader.cc:213] Running initialization op on SavedModel bundle at path: tran

In [4]:
tf.__version__

'2.11.1'

**Verifying the tmlite model using the python code**

In [19]:
# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=model_name)
interpreter.allocate_tensors()


INFO: Created TensorFlow Lite delegate for select TF ops.
2023-12-13 19:20:25.991563: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
INFO: TfLiteFlexDelegate delegate: 42 nodes delegated out of 272 nodes with 13 partitions.

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [20]:
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [21]:
# Test the model on random input data.
#input_shape = input_details[0]['shape']
input_data = ["We are the world"]
input_data = tf.constant(input_data)
interpreter.set_tensor([], input_data)

TypeError: SetTensor(): incompatible function arguments. The following argument types are supported:
    1. (self: tensorflow.lite.python.interpreter_wrapper._pywrap_tensorflow_interpreter_wrapper.InterpreterWrapper, i: int, value: handle, subgraph_index: int = 0) -> object

Invoked with: <tensorflow.lite.python.interpreter_wrapper._pywrap_tensorflow_interpreter_wrapper.InterpreterWrapper object at 0x7f3cdd1cf070>, [], <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'We are the world'], dtype=object)>